In [42]:
import mne
import os
import pickle
# os.chdir('C:\\Users\\Esimk_000\\Desktop\\School\\Spring 2018\\Cs59866\\EEG Project\\seniorprojecteeg-master-fork\\preprocessing\\Artifact_Removal')
%run preprocecssing_helpers.ipynb
%run Batch_ArtifactFilter_Epoch.ipynb
%run ../StimCodes.ipynb

import time
import mne
from mne.preprocessing import ICA
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20.0, 15.0)
matplotlib.rcParams.update({'font.size': 15})

In [20]:
# path = 'D:/EEGLAB/'
# eeglab_files = list_eeglab_files(path)

# path = '/sharedData3/proj_mdg/scratch-eproj/NEW_EPOCH_DATA/'

In [22]:
# files = os.listdir(path)
# files

['20140320_1421_epo.fif',
 '20140313_1439_epo.fif',
 '20140206_1029_epo.fif',
 '20140402_1603_epo.fif',
 '20131216_1441_epo.fif',
 '20140318_1043_epo.fif',
 '20140304_1418_epo.fif',
 '20140319_1043_epo.fif',
 '20140331_1753_epo.fif',
 '20140418_1421_epo.fif',
 '20140217_1211_epo.fif',
 '20140219_1046_epo.fif',
 '20140306_1133_epo.fif',
 '20140324_1553_epo.fif',
 '20140225_1422_epo.fif',
 '20140415_1309_epo.fif',
 '20140402_1734_epo.fif',
 '20140411_1105_epo.fif',
 '20140220_1123_epo.fif',
 '20140410_1217_epo.fif',
 '20140227_1410_epo.fif',
 '20140227_1058_epo.fif',
 '20140325_1007_epo.fif',
 '20140205_1114_epo.fif',
 '20140206_1151_epo.fif',
 '20140318_1211_epo.fif',
 '20140404_1418_epo.fif',
 '20140314_1553_epo.fif',
 '20140418_1546_epo.fif',
 '20140207_1146_epo.fif',
 '20140219_1205_epo.fif',
 '20140207_1219_epo.fif',
 '20140324_1441_epo.fif',
 '20140327_1022_epo.fif',
 '20140205_1230_epo.fif',
 '20131216_1310_epo.fif',
 '20140411_0929_epo.fif',
 '20140313_1630_epo.fif',
 '20140410_1

In [11]:
modality_lexicality_event_ids = {
   "AALL" : 112,
   "AALN" : 212,
   "AANL" : 312,
   "AANN" : 412,
   "AVLL" : 512,
   "AVNN" : 612,
   "VALL" : 712,
   "VANN" : 812,
   "VVLL" : 912,
   "VVLN" : 1012,
   "VVNL" : 1112,
   "VVNN" : 1212
}

In [1]:
# def get_epoch_trials(epoch_object):
#     #note: third stim-code has a value that is 1 more than the first.
#     events = epoch_object.events
#     found_first_stim_code = False
#     found_second_stim_code = False
#     found_third_stim_code = False
#     current_trial = []
#     trials = []
#     for event in events:
#         if not found_first_stim_code and not found_second_stim_code and not found_third_stim_code: #still searching for the end of trial
#             current_trial.append(event)
#             found_first_stim_code = True
#             first_stim_code = event
#         elif not found_second_stim_code and not found_third_stim_code:
#             current_trial.append(event)
#             found_second_stim_code = True
#         elif not found_third_stim_code and (event[-1] + 1) == first_stim_code[-1]: 
#             current_trial.append(event)
#             found_third_stim_code = True
#             trials.append(current_trial)
#             current_trial = []
#             found_first_stim_code = False
#             found_second_stim_code = False
#             found_third_stim_code = False
#         elif not found_third_stim_code and (event[-1] + 1) != first_stim_code[-1]:
#             # there is no third event, trial is unfinished.
#             current_trial = []
#             current_trial.append(event)#assuming it's first stim-code...for now
#             found_first_stim_code = True
#     return trials

In [ ]:
congruency_modality_lexicality_event_ids = {
   "AALL-c" : 1120,
   "AALL-nc" : 1121, 
   "AALN-c" : 2120,
    "AALN-nc" : 2121,
   "AANL-c" : 3120,
    "AANL-nc" : 3121,
   "AANN-c" : 4120,
    "AANN-nc" : 4121,
   "AVLL-c" : 5120,
    "AVLL-nc" : 5121,
   "AVNN-c" : 6120,
    "AVNN-nc" : 6121,
   "VALL-c" : 7120,
    "VALL-nc" : 7121,
   "VANN-c" : 8120,
    "VANN-nc" : 8121,
   "VVLL-c" : 9120,
    "VVLL-nc" : 9121,
   "VVLN-c" : 10120,
    "VVLN-nc" : 10121,
   "VVNL-c" : 11120,
    "VVNL-nc" : 11121,
   "VVNN-c" : 12120,
    "VVNN-nc" : 12121
    
}

In [62]:
def set_congruenct_labels(original_events, epoch_object):
    """
    Description:
        This method only works if the original_events object is from the epoch_object. 
    """
    congruency_event_id ={ 133 : 'congruent' , 134 : 'incongruent'}
    concruency_labels ={'congruent': 0, 'incongruent': 1}
    
    new_events = []
    epoch_events = epoch_object.events
    for event_row in original_events:
        if event_row[-1]  in congruency_event_id:
            
            congruency_status = congruency_event_id[event_row[-1]]
            sample_time = event_row[0]
            print("sample time",event_row)
            for i in range(len(epoch_events[:,-1])):
#                 print(epoch_events[i,0])
                if sample_time < epoch_events[i,0] and sample_time > epoch_events[0,0]:
                    print("first less than",epoch_events[i,0])
                    print("start event row",epoch_events[i-2])
                    new_events.append([epoch_events[i-3]])
                    new_events.append([epoch_events[i-2]])
                    new_events.append([epoch_events[i-1]])
                    break
                    
    return np.concatenate(new_events)
            

In [ ]:
# with open('events_dict.pickle', 'rb') as f:
#     events_dict = pickle.load(f)

In [ ]:
# keys = events_dict.keys()
# eeglab_events = events_dict['F:/EEGLAB/SA20131216\\20131216_1441.set']
# epoch = mne.read_epochs(os.path.join(path, files[4]))

In [63]:
# new_events = set_congruenct_labels(original_events=eeglab_events, epoch_object=epoch)

sample time [6403    0  134]
sample time [8599    0  133]
first less than 9001
start event row [7615    0    3]
sample time [9768    0  133]
first less than 10086
start event row [9251    0    3]
sample time [10853     0   133]
first less than 11163
start event row [10336     0     3]
sample time [11930     0   134]
first less than 12240
start event row [11413     0     3]
sample time [13006     0   133]
first less than 13316
start event row [12489     0     3]
sample time [14083     0   134]
first less than 14401
start event row [13557     0     3]
sample time [15168     0   133]
first less than 15478
start event row [14642     0     3]
sample time [16244     0   134]
first less than 16563
start event row [15728     0     3]
sample time [17330     0   134]
first less than 17640
start event row [16813     0     3]
sample time [18406     0   134]
first less than 18716
start event row [17889     0     3]
sample time [19483     0   133]
first less than 19793
start event row [18966     0  

sample time [270016      0    134]
first less than 270367
start event row [269316      0     18]
sample time [271938      0    133]
first less than 272279
start event row [270617      0     18]
sample time [273054      0    133]
first less than 273347
start event row [272529      0     18]
sample time [274122      0    134]
first less than 274450
start event row [273597      0     18]
sample time [275290      0    134]
first less than 275681
start event row [274699      0     18]
sample time [276663      0    133]
first less than 276999
start event row [275931      0     18]
sample time [277774      0    133]
first less than 278067
start event row [277249      0     18]
sample time [278928      0    133]
first less than 279290
start event row [278317      0     18]
sample time [280167      0    134]
first less than 280513
start event row [279540      0     18]
sample time [281444      0    134]
first less than 283915
start event row [280763      0     18]
sample time [283125      0    

start event row [548838      0      3]
sample time [550536      0    134]
first less than 552291
start event row [549949      0      3]
sample time [551930      0    134]
first less than 552291
start event row [549949      0      3]
sample time [554643      0    133]
first less than 555013
start event row [552541      0      3]
sample time [556109      0    134]
first less than 557717
start event row [555254      0      3]
sample time [558703      0    134]
first less than 559052
start event row [557976      0      3]
sample time [559827      0    133]
first less than 560129
start event row [559311      0      3]
sample time [560904      0    133]
first less than 561205
start event row [560387      0      3]
sample time [561980      0    134]
first less than 562299
start event row [561464      0      3]
sample time [563074      0    134]
first less than 563384
start event row [562557      0      3]
sample time [564159      0    133]
first less than 564469
start event row [563634      0

start event row [1094091       0      15]
sample time [1096133       0     134]
first less than 1096443
start event row [1095607       0      15]
sample time [1097383       0     134]
first less than 1098001
start event row [1096684       0      15]
sample time [1098923       0     134]
first less than 1099276
start event row [1098243       0      15]
sample time [1100295       0     134]
first less than 1101490
start event row [1099526       0      15]
sample time [1102795       0     133]
first less than 1103143
start event row [1101739       0      15]
sample time [1103910       0     133]
first less than 1105288
start event row [1103384       0      15]
sample time [1106254       0     134]
first less than 1106605
start event row [1105537       0      15]
sample time [1107451       0     133]
first less than 1122384
start event row [1106855       0      15]
sample time [1123612       0     133]
first less than 1124864
start event row [1122633       0      21]
sample time [1125878  

In [66]:
# new_events.shape

(1914, 3)

In [67]:
# epoch.events.shape

(1753, 3)

In [92]:
# filtered_epoch = epoch[(epoch.events[:,-1] == 3) | (epoch.events[:,-1] == 4)]

In [93]:
# filtered_epoch.events

array([[  7615,      0,      3],
       [  9251,      0,      3],
       [ 10336,      0,      3],
       [ 11413,      0,      3],
       [ 12489,      0,      3],
       [ 13557,      0,      3],
       [ 14642,      0,      3],
       [ 15728,      0,      3],
       [ 16813,      0,      3],
       [ 17889,      0,      3],
       [ 18966,      0,      3],
       [ 20043,      0,      3],
       [ 21696,      0,      3],
       [ 22773,      0,      3],
       [ 23849,      0,      3],
       [ 24934,      0,      3],
       [ 26011,      0,      3],
       [ 27639,      0,      3],
       [ 28707,      0,      3],
       [ 47964,     13,      4],
       [157172,      0,      4],
       [159239,      0,      4],
       [160824,      0,      4],
       [161892,      0,      4],
       [162968,      0,      4],
       [164148,      0,      4],
       [165216,      0,      4],
       [166284,      0,      4],
       [167352,      0,      4],
       [169626,      0,      4],
       [17

In [44]:
# np.concatenate(np.array(new_events))

array([1317252,       0,       2, ..., 1316692,       0,       1])

In [9]:
# c ={ 133 : 'congruent' , 134 : 'incongruent'}
# clist = [133,134]
# 134 in c
# c[133]

'congruent'

In [ ]:
def get_stim_code_epoch_trials(epoch_object):
    """
    Description:
        returns a numpy 3darray of trials. Checks if three consecutive events follow the format of 
        the first stim-code in the trial is equal to the last plus one. For example, if the first stim
        code is 12, then the last must be 11. If not, the method rejects that trial.
        
    Variables:
        epoch_object:
            mne.epochs object.
            
    returns:
        trials: 3d numpy array
    """
    events = epoch_object.events
    found_third_stim_code = False
    current_trial = []
    trials = []
    
    for i in range(len(events[:,-1])-2):
        found_third_stim_code = False
        while (not found_third_stim_code):
            stim_code_1 = events[i]
            stim_code_2 = events[i+1]
            stim_code_3 = events[i+2]
#             print(stim_code_1,stim_code_2,stim_code_3)
            
            if stim_code_1[-1] == stim_code_3[-1] + 1:
                trials.append([stim_code_1,stim_code_2,stim_code_3])
                found_third_stim_code = True
            else:
                
                break;
    return trials

In [ ]:
def stim_code_combination_from_trial(stim_code_tuple, stim_combinations):
    """
    Description:
        returns the correct stim-code combination event id 
        example - a trial has events (20, 15, 19), this method will match those values to a 
        stim combination that is defined in the dictionary stim_combinations
    """
    for stim_code in stim_combinations.keys():
        if stim_code_tuple == stim_code:
            return stim_combinations[stim_code]
    return "NaN"

In [5]:
def convert_epoch_events_to_stim_combinations(epoch_object):
    trials = get_stim_code_epoch_trials(epoch_object)
#     print(trials)
    new_events_master = []
    
    for trial in trials:
#         print(trial)
        trial = np.array(trial)
        stim_codes = trial[:,-1]
        try:
#             print("stim_codes",stim_codes)
            stim_code_tuple = (stim_codes[0],stim_codes[1], stim_codes[2])
#             print("stim_code_tuple",stim_code_tuple)
            stim_combination = stim_code_combination_from_trial(stim_code_tuple, stim_combinations)
#             print("stim_combination",stim_combination)
            new_event_id = modality_lexicality_event_ids[stim_combination]
#             print("New event_id", new_event_id)
            trial[:,-1] = new_event_id
            new_events_master.append(trial)
        except:
            pass
#             print("Error with stim-code: {0}".format(stim_code_tuple))
#     print(new_events_master)
    return np.concatenate([x for x in new_events_master])

In [6]:
def epoch_indexes_to_drop(epoch_object, new_events):
    """
    Description:
        Some epoch objects have events that are not in order for a trial. 
        These events must be dropped. This method figures out which indicies in the events
        numpy array need to be dropped.
    
    Variables:
        epoch_object:
            mne.epochs object
        new_events:
            3d numpy array of events that have already been converted to stim-code combinations
            use method convert_epoch_events_to_stim_combinations to get this array.
    returns:
    -------
        a list of indices to drop.
    
        
    """
    epoch_events = epoch_object.events
    indices_to_drop = []
    for i in range(len(epoch_events)):
        if epoch_events[i,0] not in new_events[:,0]:
            indices_to_drop.append(i)
    return indices_to_drop

In [ ]:
def generate_indices_tuples(indices_to_drop, len_epochs):
    """
    Description:
        To drop the epochs that correspond to bad events in an epoch object, a list of (start,end)
        tuples are required.
        
    Variables:
        indices_to_drop:
            A list of ints 
            
        len_epochs:
            the number of epochs in the epoch object that will have it's bad epochs dropped.
    returns:
    -------
        if indices_to_drop is empty, return empty list
        else return a list of tuples.
    """
    indices_tuples = []
    if indices_to_drop !=[]:
        
        for i in range(len(indices_to_drop)):
            if i == 0:
                indices_tuples.append((0,indices_to_drop[i]))
                print(indices_tuples)
            else:
                if indices_to_drop[i-1] +1 == indices_to_drop[i]:
                    pass
                else:
                    indices_tuples.append((indices_to_drop[i-1] +1, indices_to_drop[i]))
            print(indices_to_drop)

        indices_tuples.append((indices_to_drop[-1] +1, len_epochs))
        return indices_tuples
    else:
        return []
        
def drop_bad_epoch_events(epoch_object, indices_to_drop):
    """
    Description:
        drops the epochs of an epoch object that correspond to bad events that do not line up
        with a trial. For example: events in this order [4,5,3,2,4,5,3] would require the event labeled
        as "2" to be dropped because it does not line up with the trial [3,5,4].
    
    Variables:
        epoch_object:
            mne.epochs object.
        indices_to_drop:
            a list of ints that have the index of each epoch to drop.
    returns:
    -------
        epoch_object:
            mne.epochs object with the bad epochs droppped.
    
    """
    indices_tuples = generate_indices_tuples(indices_to_drop, epoch.events.shape[0])
    epoch_list = []
    if indices_tuples !=[]:
        for index in indices_tuples:
            epoch_list.append(epoch_object[index[0]:index[1]])
        if epoch_list != []:
            return mne.epochs.concatenate_epochs(epoch_list)
    else: 
        return epoch_object

In [13]:
def convert_epoch_object_events(epoch_object):
    """
    Description:
        Method converts the 1-24 stim-code events of an epoch object to stim-code
        combinations. Given the following sequence of events [4,5,3,11,6,10], they will be 
        considered in groups of three. The first group is (4,5,3) and the second is (11,6,10).
        These two are also considered trials. They are then matched with a dictionary called 
        "stim_combinations" found in ../preprocessing/StimCodes and the new sequence will 
        become [712,712,712,212,212,212]. The groups belong to trails and therefore
        must be given the same label for each item in that trial. 
        
        This method also drops the bad epochs that do not line up with trials.
    Variables:
        epoch_object:
            mne.epochs object
            
    returns:
    --------
        new_epoch:
            an mne.epochs object with events converted to stim_code combinations. 
    """
    new_events = convert_epoch_events_to_stim_combinations(epoch_object=epoch_object.copy())
    indices_to_drop = epoch_indexes_to_drop(epoch_object.copy(), new_events)
    new_epoch = drop_bad_epoch_events(epoch_object.copy(), indices_to_drop=indices_to_drop)
    new_epoch.events = new_events
    new_epoch.event_id = modality_lexicality_event_ids
    return new_epoch

In [16]:
# for epoch_file in epoch_files:
#     file = os.path.join(path, epoch_file)
#     epoch = mne.read_epochs(file, preload = True);
    
#     print("**epoch events shape", epoch.events.shape)
# #     try:
        
#     new_events = convert_epoch_events_to_stim_combinations(epoch_object=epoch.copy())
#     print("**new_events shape", new_events.shape)

#     indicies_to_drop = epoch_indexes_to_drop(epoch.copy(), new_events)
#     print("**indicies to drop", indicies_to_drop)
#     new_epoch = drop_bad_epoch_events(epoch.copy(), indicies_to_drop=indicies_to_drop)
#     print("**new epoch shape", new_epoch.get_data().shape)
# #     except:
# #         print("**Error with file {0}".format(epoch_file))

In [ ]:
# def isCorrect_stim_code_trials(trials):
#     """
#     Description:
#         A sanity check that makes sure all trials are in the correct order
#         The third stim-code needs to have a value that is one less than the 
#         first stim-code.
        
#     Variables:
#         trials: An ndarray of shape  (n_trials, 3, 3)
#     -----
#     returns:
#         boolean if all are correct, or one is incorrect. 
#         It will print all incorrect trials
#     """
#     has_incorrect_trial = True
#     for trial in trials:
#         if trial[:,-1][0] == trial[:,-1][-1] + 1:
#             pass
#         else:
#             print("Incorrect Trial \n: {0}".format(trial))
#             has_incorrect_trial = False
            
                    
#     return has_incorrect_trial

In [ ]:
# isCorrect_stim_code_trials(epoch_trials)

In [ ]:
# def create_describer_events(epoch, eeglab):
#     """
#     Description:
#         uses raw eeglab events and epoch events to return a new dictionary
#         with key the timestamp of an event, and value is the new event_id.
#         These event_ids can later be split/manipulated to label data by specific 
#         events
#     Variables:
#         epoch: mne.io.epoch object
#         raw_events: an ndarray of 3d integers shape 
        
#     returns:
#         dictionary
#     """
#     raw_events = find_events(eeglab)
#     raw_trials = get_trials(raw_events)
#     epoch_events = epoch.events
#     epoch_trials = get_epoch_trials(epoch)
    
#     for epoch_trial in epoch_trials:
#         for event in epoch_trial
#             if event[0] in 
#     #  